# Análise Comparativa de Modelos

In [1]:
from IPython.display import display, Markdown
import joblib
import numpy as np
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import ShuffleSplit, GridSearchCV, KFold, cross_validate
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

## 1. Obtenção de Dados

#### Nessa etapa obtemos novamente os arquivos brutos de dados e o dicionário antes de iniciar o pre-processamento.

In [2]:
df = pd.read_csv("../data/raw/data.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,survived,se o individuo esta vivo ou morto,qualitativa,nominal
1,pclass,Classe de ingresso,qualitativa,ordinal
2,sex,sexo,qualitativa,nominal
3,age,idade em anos,quantitativa,discreta
4,sibsp,# de irmãos/cônjuges a bordo do Titanic,quantitativa,discreta
5,parch,"# de pais/crianças a bordo do Titanic, 0 para ...",quantitativa,discreta
6,fare,Tarifa de passageiro,quantitativa,continua
7,embarked,Ponto de embarcacao,qualitativa,nominal
8,class,Se refere a classe social ou categoria que os ...,qualitativa,ordinal
9,who,informacao do genero (desnecessaria pois ja te...,qualitativa,nominal


---

## 2. Preparação de dados

In [3]:
# Remover as colunas indesejadas
columns_to_remove = ['who', 'deck', 'alive', 'embarked', 'class']
df = df.drop(columns=columns_to_remove, axis=1)

# Atualizar o dicionário de variáveis se necessário
# Por exemplo, se o df_dict tiver informações sobre as colunas removidas, você pode removê-las também
df_dict = df_dict[~df_dict['variavel'].isin(columns_to_remove)]

target_column = 'survived'

nominal_columns = (
    df_dict
    .query("subtipo == 'nominal' and variavel != @target_column")
    .variavel
    .tolist()
)

ordinal_columns = (
    df_dict
    .query("subtipo == 'ordinal'")
    .variavel
    .tolist()
)

discrete_columns = (
    df_dict
    .query("subtipo == 'discreta'")
    .variavel
    .tolist()
)

continuous_columns = (
    df_dict
    .query("subtipo == 'continua'")
    .variavel
    .tolist()
)
X = df.drop(columns=[target_column], axis=1)
y = df[target_column]

- Esta etapa organiza os dados para a modelagem, separando a variável-alvo das variáveis independentes e categorizando cada variável conforme seu tipo ``(nominal, ordinal, discreta, contínua)``. Isso permite aplicar pré-processamentos específicos, como **codificação** para variáveis nominais e **normalização** para variáveis contínuas, garantindo que o modelo de machine learning receba dados limpos e prontos para análise.
- Essa etapa também é responsável por remover colunas redundantes ou que não seriam interessantes para o modelo interpretar.

In [4]:
num_variables = len(df.columns)
print(f"Número total de variáveis: {num_variables}")

Número total de variáveis: 10


---

In [5]:
# Remover outliers na variável 'fare'
Q1 = df['fare'].quantile(0.25)
Q3 = df['fare'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df['fare'] = df['fare'].clip(lower=lower_bound, upper=upper_bound)


nominal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('encoding', OneHotEncoder(sparse_output=False, drop='first')),  # Codificação de variáveis
    ('normalization', StandardScaler())  # Normalização de dados
])
continuous_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='mean')),  # Tratamento de dados faltantes
    ('normalization', StandardScaler())  # Normalização de dados
])
ordinal_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('encoding', OrdinalEncoder())  # Codificação ordinal
])
discrete_preprocessor = Pipeline([
    ('missing', SimpleImputer(strategy='most_frequent')),  # Tratamento de dados faltantes
    ('normalization', StandardScaler())  # Normalização de dados
])
preprocessor = ColumnTransformer([
    ('nominal', nominal_preprocessor, nominal_columns),
    ('continuous', continuous_preprocessor, continuous_columns),
    ('ordinal', ordinal_preprocessor, ordinal_columns),
    ('discrete', discrete_preprocessor, discrete_columns)
])
X_preprocessed = continuous_preprocessor.fit_transform(df[continuous_columns])
model = LogisticRegression()

final_pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('model', model)
])


- Este código remove outliers da variável `fare` ajustando seus valores dentro de um intervalo baseado no IQR. Em seguida, ele define pipelines para diferentes tipos de variáveis: o pipeline nominal lida com valores faltantes, faz codificação one-hot e normaliza os dados; o pipeline contínuo imputa a média e normaliza; o pipeline ordinal faz imputação e codificação ordinal; e o pipeline discreto faz imputação e normalização. Esses pipelines são combinados em um `ColumnTransformer`, que os aplica às colunas correspondentes. O pipeline final inclui esse pré-processamento e um modelo de Regressão Logística.

---

## 3. Seleção de Modelos

##### Para o conjunto de dados do Titanic, o objetivo é prever a sobrevivência dos passageiros, um problema de classificação binária. Iremos analisar quatro modelos de aprendizado de máquina que serão testados utilizando um método de validação para garantir a melhor configuração possível para cada um deles. Os modelos escolhidos são:

- K-Nearest-Neighbors
- Gradient Boosting
- Decision Tree
- Random Forest

##### Cada um desses algoritmos será testado com diferentes hiperparâmetros, utilizando técnicas como Grid Search ou Random Search para otimização. O objetivo é encontrar o modelo que melhor se adapta ao conjunto de dados do Titanic, levando em consideração a complexidade e a capacidade preditiva.

##### Utilizaremos as seguintes métricas:

- **Acurácia** (Accuracy): Proporção de passageiros corretamente classificados como sobreviventes ou não sobreviventes em relação ao total de passageiros.
- **Precisão** (Precision): Proporção de passageiros corretamente previstos como sobreviventes em relação ao total de passageiros previstos como sobreviventes..
- **Recall**: Proporção de passageiros corretamente previstos como sobreviventes em relação ao total de passageiros que realmente sobreviveram.
- **F1-Score**: A média harmônica entre precisão e recall, proporcionando uma métrica equilibrada que leva em conta tanto falsos positivos quanto falsos negativos.

In [6]:
# experiment settings
n_splits_comparative_analysis = 10
n_folds_grid_search = 5
test_size = .2
random_state = 42
scoring = 'accuracy'
metrics = ['accuracy', 'precision_macro', 'recall_macro', 'f1_macro']

# model settings
max_iter = 1000
models = [
    ('K-Nearest Neighbors', KNeighborsClassifier(), {"n_neighbors": range(3, 20, 2), 'weights': ['uniform', 'distance']}),
     ('Gradient Boosting', GradientBoostingClassifier(random_state=random_state), {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7]
    }),
    ('Decision Tree',  DecisionTreeClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8]}),
    ('Random Forest',  RandomForestClassifier(random_state=random_state), {'criterion':['gini','entropy'],'max_depth': [3, 6, 8], 'n_estimators': [10, 30]}),
]

In [19]:
results = pd.DataFrame({})
cross_validate_grid_search = KFold(n_splits=n_folds_grid_search)
cross_validate_comparative_analysis = ShuffleSplit(n_splits=n_splits_comparative_analysis, test_size=test_size, random_state=random_state)
for model_name, model_object, model_parameters in models:
    print(f"running {model_name}...")
    model_grid_search = GridSearchCV(
        estimator=model_object,
        param_grid=model_parameters,
        scoring=scoring,
        n_jobs=-1,
        cv=cross_validate_grid_search
    )
    approach = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model_grid_search)
    ])
    scores = cross_validate(
        estimator=approach,
        X=X,
        y=y,
        cv=cross_validate_comparative_analysis,
        n_jobs=-1,
        scoring=metrics
    )
    scores_df = pd.DataFrame(scores)
    aggregated_scores = scores_df.agg(['mean', 'std'])
    aggregated_scores['model_name'] = model_name
    display(aggregated_scores)
    results = pd.concat([results, aggregated_scores], ignore_index=True)


running K-Nearest Neighbors...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model_name
mean,1.616019,0.039787,0.814525,0.810565,0.793219,0.798761,K-Nearest Neighbors
std,0.397508,0.008835,0.040867,0.044044,0.045827,0.045516,K-Nearest Neighbors


running Gradient Boosting...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model_name
mean,46.402391,0.025993,0.832402,0.830733,0.812697,0.818565,Gradient Boosting
std,1.778310,0.006415,0.024280,0.028270,0.027969,0.027656,Gradient Boosting


running Decision Tree...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model_name
mean,0.223169,0.028916,0.837430,0.833512,0.819371,0.824668,Decision Tree
std,0.049917,0.005707,0.026526,0.026516,0.031544,0.029792,Decision Tree


running Random Forest...


,fit_time,score_time,test_accuracy,test_precision_macro,test_recall_macro,test_f1_macro,model_name
mean,4.386272,0.027492,0.829050,0.829415,0.806322,0.813350,Random Forest
std,1.055941,0.006893,0.027646,0.028862,0.033995,0.032414,Random Forest


In [21]:
def highlight_best(s, props=''):
    if s.name[1] != 'std':
        if s.name[0].endswith('time'):
            return np.where(s == np.nanmin(s.values), props, '')
        return np.where(s == np.nanmax(s.values), props, '')

display(Markdown("### 3.1 Resultados gerais"))
(
    results
    .groupby('model_name')
    .agg(['mean', 'std']).T
    .style
    .apply(highlight_best, props='color:white;background-color:gray;font-weight: bold;', axis=1)
    .set_table_styles([{'selector': 'td', 'props': 'text-align: center;'}])
)

### 3.1 Resultados gerais

Como pode ser visto, o classificador `Decision Tree` obteve melhores resultados para praticamente todas as métricas, portanto, podemos obter os melhores parâmetros deste modelo e salvá-lo em disco para utilização em uma próxima etapa.

In [22]:
# Obtém o modelo e os parâmetros para a Árvore de Decisão
model_name, model_object, model_parameters = [foo for foo in models if foo[0] == "Decision Tree"][0]

# Configura o GridSearchCV para o modelo de Árvore de Decisão
model_grid_search = GridSearchCV(
    estimator=model_object,
    param_grid=model_parameters,
    scoring=scoring,
    n_jobs=-1,
    cv=cross_validate_grid_search
)

# Cria o pipeline com o pré-processador e o modelo
approach = Pipeline([
    ("preprocessor", preprocessor),
    ("model", model_grid_search)
])

# Ajusta o pipeline aos dados
approach.fit(X, y)  # Seleciona o melhor modelo

# Imprime os hiperparâmetros do modelo que obtiveram o melhor desempenho
print(f"Hiperparâmetros do modelo: {approach.steps[1][1].best_params_}")


Hiperparâmetros do modelo: {'criterion': 'entropy', 'max_depth': 3}


In [23]:
joblib.dump(approach, '../models/model.joblib') # Salva o modelo em disco

['../models/model.joblib']